In [2]:
from ved_src.utils import load_docs

In [3]:
nodes = load_docs('./rag/storage/docstore/article_nodes_embedded_combined')

In [4]:
from llama_index.retrievers.bm25 import BM25Retriever

In [5]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio

nest_asyncio.apply()

In [6]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().handlers = []
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import (
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
)
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings

In [7]:
import os
import pickle
import time
from typing import List

import google.generativeai as genai
from dotenv import load_dotenv
from google.generativeai import GenerationConfig
from llama_index.core.schema import TextNode
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.gemini import Gemini
from tqdm import tqdm

from src.indexing_utils import load_docs, save_docstore
from ved_src.utils import make_prompt
from src.discord_utils import send_msg


In [8]:
embed_model = GeminiEmbedding(
        model_name="models/embedding-001", api_key=os.environ.get("GOOGLE_API_KEY"))

llm = Gemini(api_key=os.environ.get("GOOGLE_API_KEY"))

Settings.embed_model = embed_model
Settings.llm = llm


In [10]:
# initialize storage context (by default it's in-memory)
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

In [ ]:
index = VectorStoreIndex(
    nodes=nodes,
    storage_context=storage_context,
    embed_model=Settings.embed_model,
)

In [ ]:
retriever = BM25Retriever.from_defaults(nodes=nodes, similarity_top_k=2)
from llama_index.core.response.notebook_utils import display_source_node

# will retrieve context from specific companies
nodes = retriever.retrieve("What is the proper way to massage")
for node in nodes:
    print(node.text)
    display_source_node(node)

Ayurveda places great importance on the birth of healthy children with sound bodies, minds, and souls. To achieve this, it provides guidelines for women from the time they reach puberty. It suggests that women should marry between the ages of 16 and 25. For conception to occur, several factors are necessary: the right time of the month (ovulation), healthy reproductive organs in the woman, and proper nourishment. During pregnancy, the woman's physical, mental, and emotional well-being undergoes significant changes. Therefore, Ayurveda emphasizes the importance of a healthy diet and lifestyle for the pregnant woman. This includes consuming wholesome foods such as rice, lentils, wheat, ghee, milk, and fruits. Additionally, Ayurveda recommends specific herbal decoctions for each month of pregnancy to support the development of the fetus. By following these guidelines, Ayurveda aims to ensure the health and well-being of both the mother and the child. Garbhin`ee Paricharyaa Ayurveda has th

**Node ID:** 9b884e0a-af15-48de-a164-7195d22b4560<br>**Similarity:** 4.934239849070215<br>**Text:** Ayurveda places great importance on the birth of healthy children with sound bodies, minds, and s...<br>

Paadaaghaata is a type of Ayurvedic massage technique that involves applying pressure with the feet. It is typically performed during the winter season as part of a daily routine. Before receiving this massage, the patient should first undergo Abhyanga (oil massage) and Mardana (muscle kneading). The person performing the massage should have clean, soft feet without any cracks and should be experienced in the technique. After the massage, the patient should bathe with warm water or a decoction. This massage technique is particularly prevalent in southern India. Prakaara Paribhaashaa: The procedure of massage with the help of feet with pressure. This should be done as a daily regime during Hemanta Ritu. Poorvakarma (Prerequisites): Patient Preparation: Paadaaghaata should be done after Abhyanga and Mardana. Pradhaanakarma: Proper and adequate physical exercise should be done and then Paadaaghaata should be done. The person who is going to do Paadaaghaata should have clean, soft feet wit

**Node ID:** b04f8f21-1d1d-4ee2-9cf7-8073c48140df<br>**Similarity:** 4.603025419613619<br>**Text:** Paadaaghaata is a type of Ayurvedic massage technique that involves applying pressure with the fe...<br>

In [ ]:
from llama_index.core.tools import RetrieverTool

vector_retriever = index.as_retriever()
bm25_retriever = BM25Retriever.from_defaults(nodes=nodes, similarity_top_k=2)

retriever_tools = [
    RetrieverTool.from_defaults(
        retriever=vector_retriever,
        description="Useful in most cases",
    ),
    RetrieverTool.from_defaults(
        retriever=bm25_retriever,
        description="Useful if searching about specific information",
    ),
]

from llama_index.core.retrievers import RouterRetriever

retriever = RouterRetriever.from_defaults(
    retriever_tools=retriever_tools,
    llm=Settings.llm,
    select_multi=False,
)

In [ ]:
# will retrieve all context from the author's life
nodes = retriever.retrieve(
    "What are various types of doshas in ayurveda"
)
for node in nodes:
    display_source_node(node)

Selecting retriever 1: Searching about specific information.


ValidationError: 1 validation error for NodeWithScore
node
  Can't instantiate abstract class BaseNode with abstract methods get_content, get_metadata_str, get_type, hash, set_content (type=type_error)

In [12]:
#mro of nodes[0]
nodes[0].__class__.__mro__

(llama_index.core.schema.TextNode,
 llama_index.core.schema.BaseNode,
 llama_index.core.schema.BaseComponent,
 pydantic.v1.main.BaseModel,
 pydantic.v1.utils.Representation,
 object)